In [1]:
# --------------------------------------------------
# 1️⃣ Mount Google Drive (optional, for cache)
# --------------------------------------------------
from google.colab import drive
import os

if not os.path.ismount("/content/drive"):
    drive.mount("/content/drive")
else:
    print("📦 Google Drive already mounted")

# --------------------------------------------------
# 2️⃣ Clone fenicsx-colab repository (idempotent)
# --------------------------------------------------
from pathlib import Path
import subprocess

REPO_URL = "https://github.com/seoultechpse/fenicsx-colab.git"
ROOT = Path("/content")
REPO_DIR = ROOT / "fenicsx-colab"

def run(cmd):
    subprocess.run(cmd, check=True)

if not REPO_DIR.exists():
    print("📥 Cloning fenicsx-colab...")
    run(["git", "clone", REPO_URL, str(REPO_DIR)])
elif not (REPO_DIR / ".git").exists():
    raise RuntimeError("Directory exists but is not a git repository")
else:
    print("📦 Repository already exists — skipping clone")

# --------------------------------------------------
# 3️⃣ Run setup_fenicsx.py IN THIS KERNEL (CRITICAL)
# --------------------------------------------------
print("🚀 Running setup_fenicsx.py in current kernel")

# ⚙️ Configuration
USE_COMPLEX = False  # <--- Set True ONLY if you need complex PETSc
USE_CLEAN = False    # <--- Set True to remove existing environment

# Build options
opts = []
if USE_COMPLEX:
    opts.append("--complex")
if USE_CLEAN:
    opts.append("--clean")

opts_str = " ".join(opts) if opts else ""

get_ipython().run_line_magic(
    "run", f"{REPO_DIR / 'setup_fenicsx.py'} {opts_str}"
)

# --------------------------------------------------
# 4️⃣ Sanity check
# --------------------------------------------------
try:
    get_ipython().run_cell_magic('fenicsx', '--info -np 4', '')
except Exception as e:
    print("⚠️ %%fenicsx magic not found:", e)

Mounted at /content/drive
📥 Cloning fenicsx-colab...
🚀 Running setup_fenicsx.py in current kernel
🔧 FEniCSx Setup Configuration
PETSc type      : real
Clean install   : False

📦 Google Drive detected — using persistent cache

🔧 Installing FEniCSx environment...

🔍 Verifying PETSc type...
✅ Installed: Real PETSc (float64)

✨ Loading FEniCSx Jupyter magic... %%fenicsx registered

✅ FEniCSx setup complete!

Next steps:
  1. Run %%fenicsx --info to verify installation
  2. Use %%fenicsx in cells to run FEniCSx code
  3. Use -np N for parallel execution (e.g., %%fenicsx -np 4)

📌 Note: Real PETSc is installed
   - Recommended for most FEM problems
   - For complex problems, reinstall with --complex

🐍 Python          : 3.11.14
📦 dolfinx         : 0.10.0
💻 Platform        : Linux-6.6.105+-x86_64-with-glibc2.35
🧵 Running as root : True

🔎 fenicsx runtime info
-----------------------
Environment        : fenicsx
micromamba         : /content/micromamba/bin/micromamba
MPI implementation : OPENM

---

In [5]:
%%fenicsx

"""
Darcy Flow Problem - DOLFINx 0.10
==================================
Models flow through porous media.

Governing equations:
  κ^(-1) * u + ∇p = 0  (Darcy's law)
  ∇·u = f              (Continuity equation)

Boundary conditions:
  p = p_D  on Γ_D (Dirichlet)
  u·n = 0  on Γ_N (Natural boundary condition)
"""

import numpy as np
from mpi4py import MPI
from dolfinx import mesh, fem, default_scalar_type
from dolfinx.fem.petsc import LinearProblem
import basix
import ufl

# 1. Create mesh (unit square)
domain = mesh.create_unit_square(MPI.COMM_WORLD, 32, 32, mesh.CellType.triangle)

# 2. Define mixed finite element space
# RT3 (velocity) + DG2 (pressure)
k_rt = 3  # Raviart-Thomas degree
k_dg = 2  # DG degree

RT = basix.ufl.element("Raviart-Thomas", domain.basix_cell(), k_rt)
DG = basix.ufl.element("DG", domain.basix_cell(), k_dg)
W_el = basix.ufl.mixed_element([RT, DG])
W = fem.functionspace(domain, W_el)

# 3. Define trial and test functions
(u, p) = ufl.TrialFunctions(W)
(v, q) = ufl.TestFunctions(W)

# 4. Physical parameters
kappa = fem.Constant(domain, default_scalar_type(1.0))  # Permeability

# 5. Define source term
x = ufl.SpatialCoordinate(domain)
f = -10.0 * ufl.exp(-((x[0] - 0.5)**2 + (x[1] - 0.5)**2) / 0.02)

# 6. Variational formulation
a = (ufl.dot(u, v) / kappa - ufl.div(v) * p - ufl.div(u) * q) * ufl.dx
L = -f * q * ufl.dx

# 7. Dirichlet boundary conditions (pressure)
# p = 0 on all boundaries
def boundary_all(x):
    return np.isclose(x[0], 0.0) | np.isclose(x[0], 1.0) | \
           np.isclose(x[1], 0.0) | np.isclose(x[1], 1.0)

# Pressure subspace (subspace 1)
W1 = W.sub(1)
dofs_p = fem.locate_dofs_topological(W1, domain.topology.dim - 1,
                                     mesh.locate_entities_boundary(domain, domain.topology.dim - 1, boundary_all))
bc_p = fem.dirichletbc(default_scalar_type(0), dofs_p, W1)

# 8. Solve the problem
problem = LinearProblem(a, L, bcs=[bc_p],
                       petsc_options={"ksp_type": "preonly",
                                     "pc_type": "lu",
                                     "pc_factor_mat_solver_type": "mumps"},
                       petsc_options_prefix="darcy")
wh = problem.solve()

# 9. Extract solution components
uh, ph = wh.split()

# 10. Output results
print(f"Velocity norm: {fem.assemble_scalar(fem.form(ufl.inner(uh, uh) * ufl.dx))**0.5:.6f}")
print(f"Pressure norm: {fem.assemble_scalar(fem.form(ph**2 * ufl.dx))**0.5:.6f}")
print(f"Velocity divergence: {fem.assemble_scalar(fem.form(ufl.div(uh)**2 * ufl.dx))**0.5:.6e}")

# 11. Save to VTX files (for visualization in ParaView)
# VTXWriter only supports Lagrange functions, so interpolation is needed
try:
    from dolfinx.io import VTXWriter

    # Interpolate to DG space (velocity is vector, pressure is scalar)
    gdim = domain.geometry.dim
    V_dg = fem.functionspace(domain, ("DG", 2, (gdim,)))  # Vector DG
    Q_dg = fem.functionspace(domain, ("DG", 2))           # Scalar DG

    uh_dg = fem.Function(V_dg)
    ph_dg = fem.Function(Q_dg)

    uh_dg.interpolate(uh)
    ph_dg.interpolate(ph)

    uh_dg.name = "velocity"
    ph_dg.name = "pressure"

    with VTXWriter(domain.comm, "darcy_velocity.bp", [uh_dg], engine="BP4") as vtx:
        vtx.write(0.0)

    with VTXWriter(domain.comm, "darcy_pressure.bp", [ph_dg], engine="BP4") as vtx:
        vtx.write(0.0)

    print("\nResults saved:")
    print("  - darcy_velocity.bp (open with ParaView)")
    print("  - darcy_pressure.bp (open with ParaView)")
except Exception as e:
    print(f"\nError saving VTX: {e}")
    print("Attempting to save in XDMF format.")

    from dolfinx.io import XDMFFile

    # Interpolate to DG space
    gdim = domain.geometry.dim
    V_dg = fem.functionspace(domain, ("DG", 2, (gdim,)))
    Q_dg = fem.functionspace(domain, ("DG", 2))

    uh_dg = fem.Function(V_dg)
    ph_dg = fem.Function(Q_dg)

    uh_dg.interpolate(uh)
    ph_dg.interpolate(ph)

    uh_dg.name = "velocity"
    ph_dg.name = "pressure"

    with XDMFFile(domain.comm, "darcy_solution.xdmf", "w") as xdmf:
        xdmf.write_mesh(domain)
        xdmf.write_function(uh_dg)
        xdmf.write_function(ph_dg)

    print("Results saved to darcy_solution.xdmf.")

print("\nComplete!")

Velocity norm: 0.283666
Pressure norm: 0.058830
Velocity divergence: 1.772454e+00

Results saved:
  - darcy_velocity.bp (open with ParaView)
  - darcy_pressure.bp (open with ParaView)

Complete!


In [6]:
!zip -r darcy_pressure.zip darcy_pressure.bp
!zip -r darcy_velocity.zip darcy_velocity.bp

from google.colab import files
files.download('darcy_pressure.zip')
files.download('darcy_velocity.zip')

  adding: darcy_pressure.bp/ (stored 0%)
  adding: darcy_pressure.bp/md.idx (deflated 52%)
  adding: darcy_pressure.bp/md.0 (deflated 64%)
  adding: darcy_pressure.bp/data.0 (deflated 76%)
  adding: darcy_pressure.bp/profiling.json (deflated 43%)
  adding: darcy_velocity.bp/ (stored 0%)
  adding: darcy_velocity.bp/md.idx (deflated 52%)
  adding: darcy_velocity.bp/md.0 (deflated 65%)
  adding: darcy_velocity.bp/data.0 (deflated 72%)
  adding: darcy_velocity.bp/profiling.json (deflated 43%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>